In [1]:
import scipy
from scipy.io import mmread
import scanpy as sc
import anndata
import pandas as pd
import numpy as np

In [2]:
def calculate_optimal_PC(adata, min_PC = 50, min_var=25, n_comps=100, use_hv=None):
    'select number of PCs based on min_PC and min_var threshold'
    import matplotlib.pyplot as plt
    import scanpy as sc
    from kneed import KneeLocator

    #calculate PCs
    sc.tl.pca(adata, n_comps=100, zero_center=True, svd_solver='arpack', random_state=0, return_info=False, use_highly_variable=use_hv, dtype='float32', copy=False, chunked=False, chunk_size=None)
    sc.pl.pca_variance_ratio(adata, log=False)

    #calculate number of PCs

    a = adata.copy()# PCs kneepoint
    x = [i for i in range(len(a.uns["pca"]["variance_ratio"]))]
    y = list(a.uns["pca"]["variance_ratio"])
    kneedle = KneeLocator(x,
                      y,
                      S=1,
                      curve='convex',
                      direction='decreasing',
                      online=False)
    kn_pc = round(kneedle.knee, 3)
    kneedle.plot_knee()
    plt.show()
    kneedle.plot_knee_normalized()
    plt.show()
    print("Kneepoint happens at PC:", kn_pc)

    exp_var = sum(adata.uns['pca']['variance_ratio'][:kn_pc])
    exp_var_test_percent = exp_var*100
    print(kn_pc,'PC explain',exp_var_test_percent, '% of variance')

    #find number of PCs explaining at least min variance
    tested_PC_number = kn_pc



    tested_PC_number = 1
    while exp_var_test_percent <min_var:
        tested_PC_number = tested_PC_number+1
        exp_var_test = sum(adata.uns['pca']['variance_ratio'][0:tested_PC_number])
        exp_var_test_percent = exp_var_test*100
        print(tested_PC_number, 'PC explain',exp_var_test_percent, '% of variance')
        if tested_PC_number == n_comps:
            break

            
    exp_var_test = sum(adata.uns['pca']['variance_ratio'][0:min_PC])
    exp_var_test_percent = exp_var_test*100
    if tested_PC_number <min_PC:
        print('setting PCs to',min_PC)
        print('variance of',min_PC,'is',exp_var_test_percent,'%')
        tested_PC_number = min_PC
        
        
    #define PC number for embeddings
    number_of_PC_used = tested_PC_number
    
    exp_var_test = sum(adata.uns['pca']['variance_ratio'][0:tested_PC_number])
    exp_var_test_percent = exp_var_test*100
    print('number of PCs for clusterings/embeddings is:', number_of_PC_used)
    print('these explain', exp_var_test_percent,'of variance') 
    
    #recalculate PCs
    sc.pp.pca(adata, n_comps=number_of_PC_used, zero_center=True, svd_solver='arpack', random_state=0, return_info=False, use_highly_variable=use_hv, dtype='float32', copy=False, chunked=False, chunk_size=None)
    sc.pl.pca_variance_ratio(adata, log=False)
    return number_of_PC_used

In [3]:
DATA_DIR = '/home/wallet/Jupyter/TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8/matrix_files/'

In [4]:
#naive data
naive_matrix_path = DATA_DIR+'gene_sorted-naivedata_scp.mtx'
naive_barcodes_path = DATA_DIR+'naivedata_scp.barcodes.csv'
naive_genes_path = DATA_DIR+'naivedata_scp.genes.csv'
naive_annotations_path = DATA_DIR + 'combinenaivedata-reprocessed-clean-detailed-annotations.tsv'
naive_save_path = DATA_DIR + 'TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-naive.h5ad'
#treated data
treated_matrix_path = DATA_DIR+'gene_sorted-treated_data_scp.mtx'
treated_barcodes_path = DATA_DIR+'treateddata_scp.barcodes.csv'
treated_genes_path = DATA_DIR+'treateddata_scp.genes.csv'
treated_annotations_path = DATA_DIR + 'combinetreateddata-reprocessed-final-annotations.tsv'
treated_save_path = DATA_DIR + 'TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-treated.h5ad'


adata_save_path = DATA_DIR + 'TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells.h5ad'

# get naive data

In [ ]:
naive_matrix = mmread(naive_matrix_path)
naive_matrix

In [ ]:
#get file encoding
!file -bi {naive_barcodes_path}

In [ ]:
naive_barcodes = pd.read_csv(naive_barcodes_path, compression='gzip')
del naive_barcodes['Unnamed: 0']
naive_barcodes.columns = ['barcodes']
naive_barcodes = naive_barcodes.set_index('barcodes')
naive_barcodes

In [ ]:
naive_genes = pd.read_csv(naive_genes_path,header=None)
naive_genes.columns= ['gene_names']
naive_genes['gene_names'] = naive_genes['gene_names'].astype(str)
naive_genes = naive_genes.set_index('gene_names')
naive_genes

In [ ]:
naive_annotations = pd.read_csv(naive_annotations_path,sep='\t')
naive_annotations = naive_annotations.drop(index=[0])
naive_annotations = naive_annotations.set_index('NAME')
naive_annotations

In [ ]:
#transform to dicts
naive_celltype_dict = naive_annotations['cell_subsets'].to_dict()
naive_pid_dict = naive_annotations['pid'].to_dict()
naive_ngenes_dict = naive_annotations['n_genes'].to_dict()

In [ ]:
naive_matrix_dense = np.array(naive_matrix.T.todense())

In [ ]:
from scipy.sparse import csr_matrix
adata_naive = anndata.AnnData(X=csr_matrix(naive_matrix.T), obs=naive_barcodes, var=naive_genes, uns=None, obsm=None, varm=None, layers=None, raw=None, 
                              shape=None, filename=None, filemode=None, asview=False, obsp=None, varp=None, 
                              oidx=None, vidx=None)
adata_naive

In [ ]:
adata_naive.obs['cell_subsets'] = pd.Categorical(adata_naive.obs_names.map(naive_celltype_dict).astype(str))
adata_naive.obs['pid'] = pd.Categorical(adata_naive.obs_names.map(naive_pid_dict).astype(str))
adata_naive.obs['n_genes'] = adata_naive.obs_names.map(naive_ngenes_dict).astype(int)
adata_naive.obs

In [ ]:
adata_naive.X

In [ ]:
set(adata_naive.obs['cell_subsets'])

In [ ]:
naive_save_path

In [ ]:
adata_naive.write(naive_save_path)
print('saved to:',naive_save_path)

# get treated data

In [ ]:
treated_matrix = mmread(treated_matrix_path)
treated_matrix

In [ ]:
treated_matrix = csr_matrix(treated_matrix.T)

In [ ]:
#get file encoding
!file -bi {treated_barcodes_path}

In [ ]:
treated_barcodes

In [ ]:
treated_barcodes = pd.read_csv(treated_barcodes_path,header=None)
treated_barcodes.columns = ['barcodes']
treated_barcodes = treated_barcodes.set_index('barcodes')
treated_barcodes

In [ ]:
treated_genes = pd.read_csv(treated_genes_path,header=None)
treated_genes.columns = ['gene_names']
treated_genes['gene_names'] = treated_genes['gene_names'].astype(str)
treated_genes = treated_genes.set_index('gene_names')
treated_genes

In [ ]:
treated_annotations = pd.read_csv(treated_annotations_path,sep='\t')
treated_annotations = treated_annotations.drop(index=[0])
treated_annotations = treated_annotations.set_index('NAME')
treated_annotations

In [ ]:
#transform to dicts
treated_celltype_dict = treated_annotations['cell_subsets'].to_dict()
treated_pid_dict = treated_annotations['pid'].to_dict()
treated_ngenes_dict = treated_annotations['n_genes'].to_dict()

In [ ]:
adata_treated = anndata.AnnData(X=treated_matrix, obs=treated_barcodes, var=treated_genes, uns=None, obsm=None, varm=None, layers=None, raw=None, 
                              shape=None, filename=None, filemode=None, asview=False, obsp=None, varp=None, 
                              oidx=None, vidx=None)
adata_treated

In [ ]:
adata_treated.obs['cell_subsets'] = pd.Categorical(adata_treated.obs_names.map(treated_celltype_dict).astype(str))
adata_treated.obs['pid'] = pd.Categorical(adata_treated.obs_names.map(treated_pid_dict).astype(str))
adata_treated.obs['n_genes'] = adata_treated.obs_names.map(treated_ngenes_dict).astype(int)
adata_treated.obs

In [ ]:
adata_treated.write(treated_save_path)
print('saved to:',treated_save_path)

In [ ]:
adata_treated.obs['treated'] = ['True']*len(adata_treated)
adata_treated.obs['treated'] =pd.Categorical(adata_treated.obs['treated'])
adata_naive.obs['treated'] = ['False']*len(adata_naive)
adata_naive.obs['treated'] =pd.Categorical(adata_naive.obs['treated'])


In [ ]:
#concatenate adatas
adata = adata_naive.concatenate([adata_treated],join='outer', batch_key='treated', batch_categories=['untreated','treated'], 
                            uns_merge=None, index_unique='-', fill_value=None)

In [ ]:
adata

In [ ]:
adata.write(adata_save_path)
print('saved to:',adata_save_path)

# explore data

In [5]:

adata_path  = DATA_DIR + 'TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells.h5ad'
raw_adata_path = '/home/wallet/Jupyter/TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8/raw_adatas/GSE202051_totaldata-final-toshare.h5ad'
adata_immune_save_path = DATA_DIR +  'TIL-X-PDAC-X-snRNAseq-X-2022-X-Hwang-X-10.1038_s41588-022-01134-8-X-all_cells.h5ad'

In [6]:
adata = sc.read(raw_adata_path)
adata

AnnData object with n_obs × n_vars = 224988 × 22164
    obs: 'sampleid', 'n_genes', 'n_counts', 'scrublet_scores', 'pid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_counts_sat', 'n_genes_sat', 'percent_mito', 'leiden', 'batch', 'Moffitt_basal', 'Moffitt_classical', 'Bailey_squamous', 'Bailey_progenitor', 'Collison_QM', 'Collison_classical', 'MALIGNANT CELLS', 'ACINAR', 'Alpha', 'Beta', 'Delta', 'Gamma', 'Episilon', 'ENDOCRINE', 'ENDOTHELIAL', 'Pan_Immune', 'AntigenPresentingCells', 'Monocytes_1', 'Monocytes_2', 'Macrophage', 'cDC1', 'cDC2', 'DC_activated', 'pDC', 'Mast', 'Eosinophils', 'Neutrophils', 'M0', 'M1', 'M2', 'Mast_Resting', 'Mast_activated', 'CD8_Tcells', 'CD4_Tcells', 'NK', 'CD8_gammadelta', 'CD8_exhausted', 'CD4_naive', 'CD4_memory_resting', 'CD4_memory_activated', 'CD4_follicular_helper', 'CD4_regulatory', 'NK_resting', 'NK_activated', 'B_cell', 'Plasma', 'Bcell_naive', 'Bcell_memory', 'IMMUNE', 'PanCAF', 'iCAF', 'myCAF', 'apCA

In [7]:
list(set(adata.obs['Level 1 Annotation']))[0:30]

['Adipocyte',
 'Cancer-associated fibroblast',
 'Lymphoid',
 'Vascular smooth muscle',
 'Endocrine',
 'Schwann',
 'Epithelial (malignant)',
 'Epithelial (non-malignant)',
 'Endothelial',
 'Pericyte',
 'Intra-pancreatic neurons',
 'Myeloid']

In [8]:
list(set(adata.obs['Level 2 Annotation']))[0:30]

['Delta',
 'Gamma',
 'B',
 'myCAF',
 'Natural killer',
 'CAF',
 'Intra-pancreatic neurons',
 'Adipocyte',
 'Hormone-negative neuroendocrine',
 'Acinar',
 'Beta',
 'Schwann',
 'Malignant',
 'Alpha',
 'Pericyte',
 'Ductal (atypical)',
 'Ductal',
 'Lymphatic',
 'Plasma',
 'Vascular',
 'CD8+ T',
 'ADM',
 'Vascular smooth muscle',
 'Dendritic',
 'Treg',
 'Epsilon',
 'CD4+ T',
 'Mast',
 'Macrophage',
 'Neutrophil']

In [9]:
list(set(adata.obs['Level 3 Annotation']))[0:30]

['Delta',
 'Gamma',
 'B',
 'Dendritic (activated)',
 'myCAF',
 'Natural killer',
 'CAF',
 'Intra-pancreatic neurons',
 'CD8+ T (terminally dysfunctional)',
 'Adipocyte',
 'Hormone-negative neuroendocrine',
 'CD8+ T (dysfunctional)',
 'Acinar',
 'Beta',
 'Schwann',
 'Dendritic (plasmacytoid)',
 'Malignant',
 'Alpha',
 'Acinar (REG+)',
 'Pericyte',
 'Ductal (atypical)',
 'Ductal',
 'Lymphatic',
 'CD8+ T (progenitor dysfunctional)',
 'Plasma',
 'Vascular',
 'CD8+ T',
 'ADM',
 'Vascular smooth muscle',
 'Dendritic (conventional type 2)']

In [10]:
list(set(adata.obs['Level 3 Annotation']))[30:]

['Treg',
 'Epsilon',
 'CD4+ T',
 'Mast',
 'Dendritic (conventional type 1)',
 'Treg (activated)',
 'Macrophage',
 'Neutrophil',
 'Treg (quiescent)']

In [11]:
set(adata.obs['pid'])

{'T1',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19',
 'T2',
 'T20',
 'T21',
 'T22',
 'T23',
 'T24',
 'T25',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'U1',
 'U10',
 'U11',
 'U12',
 'U13',
 'U14',
 'U15',
 'U16',
 'U17',
 'U18',
 'U2',
 'U3',
 'U4',
 'U5',
 'U6',
 'U7',
 'U8',
 'U9'}

In [13]:
import math
def revert_normalization(adata,library_size,log_base=math.e,pseudocount=1,scale_factor=10000):
    '''
    revert log1p(library_size) normalization when log base, pseudocount, library size and scaling factor is known
    adata: anndata, with normalized data in adata.X
    library_size: str, key for library size per cell stored in adata.obs
    log_base: float, base of logarithm with which data has been transformed
    pseudocount: float, pseudocount added in log1p transformation
    size_factor: float, sizing factor cell library sizes have been scaled to
    '''
    import numpy as np
    import scanpy as sc
    import scipy
    if isinstance(adata.X,scipy.sparse.csr_matrix):
        normalized = np.array(adata.X.todense())
    else:
        normalized = np.array(adata.X)
    removed_log = np.power(log_base,normalized) - pseudocount
    removed_scale = removed_log/scale_factor
    original_libsize = removed_scale.T * list(adata.obs[library_size].astype(float))
    original_libsize = original_libsize.T
    return original_libsize



In [14]:
X= revert_normalization(adata,'n_counts',log_base=2,pseudocount=1,scale_factor=10000)
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
#did not go forward with reverting normalization because library sizes seem to be incorrect --> result is not an integer
np.max(X)

210.5999454855919